# **Morphological operations**

In this notebook, we demonstrate the effects of morphological operations.

Morphological operations are basic image operations that process images based on shapes. A morphological operation 
requires two inputs: an input image and a structuring element. The structuring element is a (usually binary) image that specifies 
the neighborhood used to process each pixel in the input image. It represents the shape of the operation to be performed. 
The illustration below shows common structuring elments. 

![Common structuring elements](../data/doc/structuring-elements-2d.svg)  
**Figure 1**: Common structuring elements.

Similar to the kernels in a convolution, the structuring element is swept across the input image. However, the operations performed are different. Instead of multiplying and summing the pixel values, the morphological operations are based on set theory and logical operations (intersection, union, complement, etc.). 

For instance, the **erosion** operation (see below) computes the intersection of the image and the shifted structuring element: Only if all forground pixels (value=1) in the structuring element at a certain position overlap with forground pixels in the input image, the pixel value in the output image is set to 1 for this position, see Figure 2 Likewise, the **dilation** operation computes the union of the image and the shifted structuring element: If at least one forground pixel in the structuring element overlaps with a forground pixel in the input image, the pixel value in the output image is set to 1 for this position, see Figure 3.

![Binary erosion](../data/doc/binary-erosion.svg)  
**Figure 2**: Binary erosion of an input image (1) using a circular structuring element (2). Subfigures (3) and (4) show the result.

![Binary dilation](../data/doc/binary-dilation.svg)  
**Figure 2**: Binary dilation of an input image (1) using a circular structuring element (2). Subfigures (3) and (4) show the result.

Morphological operations are used for a variety of image processing tasks, such as removing noise, isolating individual elements, and enhancing features. Here, we will cover some elementary morphological operations: dilation, erosion, opening, and closing. Instead of providing formal definitions of these operations, we will explore them through examples.


Credits: This notebook follows a tutorial from pyimagesearch, written by Adrian Rosebrock. [Link](https://pyimagesearch.com/2021/04/28/opencv-morphological-operations/)

---

## **Preparations**

The usual preparations... Before we begin, let's load some drawing functions for rendering images effortlessly in this Jupyter notebook.

In [ ]:
import sys
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Functionality related to this course
sys.path.append("..")
import tools as isp

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

---

## **Examplary image data**

In the following we use a (binary) image displaying some text. Furthermore, we add some noise to the image.

Recall that a binary image is an image with only two possible values for each pixel. Depending on the image type, these values are often 0 and 1 (`dtype=bool`), or 0 and 255 (`dtype=np.uint8`). As the morphological operations provided by OpenCV assume images of `dtype=np.uint8`, we go with the latter option. 

Note that the morphological operations are also defined for grayscale and even color images, but for now let's assume binary images.


In [ ]:
def sample_border(w, h, margin, inside=False):
    while True:
        x = np.random.randint(0, w)
        y = np.random.randint(0, h)
        is_valid = ((x < margin) or (x >= (w - margin)) 
                    or (y < margin) or (y >= h - margin))
        if inside:
            is_valid = not is_valid
        if is_valid:
            return x, y

# Read in the image and invert it
img = cv.imread("../data/images/word-ice-cream.png", cv.IMREAD_GRAYSCALE)
img = 255 - img

# Add dots of noise such that they do not overlap with the text.
h, w = img.shape
img_noisy_w = img.copy()  # Image with white spots
img_noisy_b = img.copy()  # Image with black spots
np.random.seed(1)
for i in range(50):
    x, y = sample_border(w, h, 20)
    r = np.random.randint(1, 4)
    cv.circle(img_noisy_w, (x, y), r, 255, -1, lineType=cv.LINE_AA)
for i in range(400):
    x, y = sample_border(w, h, 20, inside=True)
    cv.circle(img_noisy_b, (x, y), 1, 0, -1, lineType=cv.LINE_AA)

# Display the image
isp.show_image_chain([img, img_noisy_w, img_noisy_b], 
                     titles=["Input image", 
                             "Image with white spots", 
                             "Image with black spots"], 
                     suppress_info=True);



---

## **Erosion**

Just like the name suggests, it erodes the image. Erosion is useful for removing small blobs in an image or by thinning the visible structures.

Erosion works by defining a structuring element and then sliding this structuring element from left-to-right and top-to-bottom across the input image. A foreground pixel in the input image will be kept only if all pixels inside the structuring element are > 0. Otherwise, the pixels are set to 0 (i.e., background). If the image was grayscale, the minimum value would be taken instead of 0.

Note how the small white spots are removed by the erosion operation and the letters are thinned.

In [ ]:
# Note: We can specify our own structuring element (SE), 
# but we can also use the default one (3x3 square).
#se = np.ones((5, 5), np.uint8)
se = None

results = {}
results["Original"] = img_noisy_w
for i in range(1, 4):
    result = cv.erode(img_noisy_w, se, iterations=i)
    results[f"Eroded ({i} iterations)"] = result
    
isp.show_image_grid(results, suppress_info=True, ncols=1, figsize=(6, 7));
#isp.save_figure(path="morpho-erosion.png")

---

## **Dilation**

Dilation is the opposite of erosion. In contrast to erosion, dilation expands the boundaries of the foreground object in an image. Dilations increase the size of foreground objects and are especially useful for joining broken parts of an image together.

Dilations also utilize structuring elements ("kernels"). A center pixel is set to white if at least one pixel in the structuring element is > 0. (If the image is grayscale, the maximum value is taken.)

In [ ]:
se = None  # Default structuring element, but you can specify your own
results = {}
results["Original"] = img_noisy_b
for i in range(1, 4):
    results[f"Dilated ({i} iterations)"] = cv.dilate(img_noisy_b, se, iterations=i)
    
isp.show_image_grid(results, suppress_info=True, ncols=1, figsize=(6, 7));
#isp.save_figure(path="morpho-dilation.png")

Note that erosion and dilation are dual operations: If we apply erosion to the inverted image, we get the same result as applying dilation to the original image. The following code illustrates this behavior:

In [ ]:
se = np.ones((5, 5), np.uint8)
img_dilated = cv.dilate(img, se, iterations=1)
img_eroded = 255 - cv.erode(255-img, se, iterations=1)
img_diff = img_dilated - img_eroded
print("Test: The difference between dilated image and eroded complement of the image should be zero:")
print("      Measured difference = %d" % np.abs(img_diff).sum())

---

## **Opening**

An opening is an erosion followed by a dilation.

Performing an opening operation allows us to remove small blobs from an image, without affecting the overall shape of the larger blobs too much: First an erosion is applied to remove the small blobs, then a dilation is applied to regrow the size of the original object.


In this example, we constructed our own structuring element. OpenCV provides a function for this:
`cv.getStructuringElement(type, size)` where `type` is one of `cv.MORPH_RECT`, `cv.MORPH_CROSS`, `cv.MORPH_ELLIPSE`,
and `size` is the size of the structuring element. A structuring element is similar to a convolution kernel, but it is used for morphological operations which have a different operation logic than kernels used in a convolution operation.

Note how we can remove the white spots with a dilation operation. The larger the kernel, the larger the spots can be removed. At the same time, the shape of the mask may brake in thin places.




In [ ]:
sizes = [(3, 3), (5, 5), (7, 7)]
results = {}
results["Original"] = img_noisy_w
for i, size in enumerate(sizes):
    # Construct a rectangular structuring elmenet from the current size and then
    # apply an "opening" operation
    se = cv.getStructuringElement(cv.MORPH_ELLIPSE, size)
    ret = cv.morphologyEx(img_noisy_w, cv.MORPH_OPEN, se, iterations=1)
    results["Opening (structuring element: %dx%d)" % size] = ret
    
isp.show_image_grid(results, suppress_info=True, ncols=1, figsize=(6, 7));
#isp.save_figure(path="morpho-opening.png")

---

## **Closing**

The opposite to an opening would be a closing. A closing is a dilation followed by an erosion. 

As the name suggests, a closing is used to close holes inside of objects or for connecting components together. We can use the same code as above, and just need to change the type of operation.


In [ ]:
sizes = [(3, 3), (5, 5), (7, 7)]
results = {}
results["Original"] = img_noisy_b
for size in sizes:
    # Construct a rectangular structuring element from the current size and then
    # apply an "opening" operation
    se = cv.getStructuringElement(cv.MORPH_ELLIPSE, size)
    ret = cv.morphologyEx(img_noisy_b, cv.MORPH_CLOSE, se, iterations=1)
    results["Closing (structuring element: %dx%d)" % size] = ret
    
isp.show_image_grid(results, suppress_info=True, ncols=1, figsize=(6, 7));
#isp.save_figure(path="morpho-closing.png")

---

## **Morphological gradient**

A morphological gradient is the difference between a dilation and erosion. It is useful for determining the outline of a particular object of an image.

We still use the same code as above.

In [ ]:
sizes = [(3, 3), (5, 5), (7, 7)]
results = {}
results["Original"] = img
for size in sizes:
    # Construct a rectangular structuring element from the current size and then
    # apply an "opening" operation
    se = cv.getStructuringElement(cv.MORPH_RECT, size)
    ret = cv.morphologyEx(img, cv.MORPH_GRADIENT, se, iterations=1)
    results["Gradient (structuring element: %dx%d)" % size] = ret
    
isp.show_image_grid(results, suppress_info=True, ncols=1, figsize=(6, 7));
#isp.save_figure(path="morpho-closing.png")

---

## **Morphological operations and grayscale images**

Up until this point we have applied morphological operations only to binary images. But the above operations are also defined for grayscale (and even color) images. 

Let's have look at an MRI image from the brain. (Source: [Radiopaedia](https://radiopaedia.org/cases/normal-brain-mri-6))


In [ ]:
img_mri = cv.imread("../data/images/brain-mri/brain-mri-a-t2-dark.jpg", cv.IMREAD_GRAYSCALE)

size = (3, 3)
se = cv.getStructuringElement(cv.MORPH_RECT, size)
erode = cv.morphologyEx(img_mri, cv.MORPH_ERODE, se, iterations=5)
dilate = cv.morphologyEx(img_mri, cv.MORPH_DILATE, se, iterations=5)
opening = cv.morphologyEx(img_mri, cv.MORPH_OPEN, se, iterations=5)
closing = cv.morphologyEx(img_mri, cv.MORPH_CLOSE, se, iterations=5)
gradient = cv.morphologyEx(img_mri, cv.MORPH_GRADIENT, se, iterations=3)

size = (11, 11)
se = cv.getStructuringElement(cv.MORPH_RECT, size)
tophat = cv.morphologyEx(img_mri, cv.MORPH_TOPHAT, se, iterations=1)

size = (25, 25)
se = cv.getStructuringElement(cv.MORPH_RECT, size)
blackhat = cv.morphologyEx(img_mri, cv.MORPH_BLACKHAT, se, iterations=1)

results = {
    "Original": img_mri,
    "Erosion": erode,
    "Dilation": dilate,
    "Opening": opening,
    "Closing": closing,
    "Gradient": gradient,
    "Top-hat": tophat,
    "Black-hat": blackhat
}

isp.show_image_grid(results, suppress_info=True, ncols=3, figsize=(9, 7));
#isp.save_figure(path="morpho-closing-grayscale.png")

## **Top-hat (white-hat) and bottom-hat (black-hat)**

Both the top-hat (white-hat) and the bottom-hat (black-hat) operators are more suited for grayscale images rather than binary ones.

A top-hat (also known as a white-hat) operation is the difference between the original (grayscale/single channel) input image and the opening.

A top-hat operation is used to reveal bright regions of an image on dark background, under the restriction that these objects are smaller than the structuring element. In contrast, a black-hat operation is used to reveal dark regions on bright backgrounds. (However, these operations are only useful if the structures of interest are significantly different in size compared to the structuring element.)

A common application for the top-hat operation is to handle uneven illumination in images and to emphasize certain structures of interest.




In [ ]:
h, w = img_mri.shape
x, y = np.meshgrid(np.arange(w), np.arange(h))
gradient = (x - w//2)**2 + (y - h//2)**2
gradient = gradient / gradient.max()
gradient = (gradient * 200).astype(np.uint8)
img_dist = (img_mri//2 + gradient//2)

In [ ]:
sizes = []
sizes.append((3,11))
sizes.append((11,3))
sizes.append((50,100))
results = {}
results["Original"] = img_mri
results["Distorted"] = img_dist
results["Dummy"] = None  # Placeholder (just for visualization)
for size in sizes:
    se = cv.getStructuringElement(cv.MORPH_RECT, size)
    ret = cv.morphologyEx(img_dist, cv.MORPH_TOPHAT, se, iterations=1)
    label = "Top-hat (kernel: %dx%d)" % size
    results[label] = ret

isp.show_image_grid(results, suppress_info=True, ncols=3, figsize=(9, 7));